In [1]:
from dotenv import load_dotenv
# Path to env file containing the waterbodies database credentials
# Only necessary on the Sandbox.
dotenv_path = "/home/jovyan/.env"
load_dotenv(dotenv_path=dotenv_path, verbose=True, override=True)

True

In [2]:
import json
import logging
import sys

import click
from datacube import Datacube
from odc.stats.model import DateTimeRange

from waterbodies.hopper import create_tasks_from_scenes
from waterbodies.io import check_directory_exists, find_geotiff_files
from waterbodies.logs import logging_setup
from waterbodies.text import format_task, get_tile_id_tuple_from_filename

In [3]:
verbose =1
run_type = "backlog-processing"
temporal_range = "2016-04-05--P1D"
# historical_extent_rasters_directory = "s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/historical_extent_rasters/"
historical_extent_rasters_directory = "../../../tests/data/"

In [4]:
# Set up logging.
logging_setup(verbose)
_log = logging.getLogger(__name__)

In [5]:
if not check_directory_exists(path=historical_extent_rasters_directory):
    e = FileNotFoundError(f"Directory {historical_extent_rasters_directory} does not exist!")
    _log.error(e)
    raise e
else:
    historical_extent_rasters = find_geotiff_files(directory_path=historical_extent_rasters_directory)

In [6]:
# Get the tile_ids for tiles that actually contain waterbodies.
tile_ids_of_interest = [
        get_tile_id_tuple_from_filename(file_path=raster_file)
        for raster_file in historical_extent_rasters
    ]

In [7]:
product = "wofs_ls"

In [8]:
# Parse the temporal range
temporal_range_ = DateTimeRange(temporal_range)

In [9]:
# Connect to the datacube
dc = Datacube(app=run_type)

In [10]:
# Define the datacube query 
dc_query = dict(product=product, time=(temporal_range_.start, temporal_range_.end))
dc_query

{'product': 'wofs_ls',
 'time': (datetime.datetime(2016, 4, 5, 0, 0),
  datetime.datetime(2016, 4, 5, 23, 59, 59, 999999))}

In [11]:
%%time
# Query the datacube for all wofs_ls datasets whose acquisition times fall within
# the temporal range specified.
scenes = dc.find_datasets(**dc_query)

CPU times: user 86.7 ms, sys: 13.4 ms, total: 100 ms
Wall time: 39.6 s


In [12]:
%%time
tasks = create_tasks_from_scenes(scenes=scenes, tile_ids_of_interest=tile_ids_of_interest)

Processing      174 scenes: 100%|██████████| 174/174 [00:00<00:00, 364.49it/s]

[2024-04-08 13:25:01,814] {hopper.py:167} INFO - Filter the 985 cells to keep only the cells containing the 1 tile ids of interest.
[2024-04-08 13:25:01,816] {hopper.py:174} INFO - Total number of cells after filtering: 1
[2024-04-08 13:25:01,817] {hopper.py:178} INFO - For each cell, group the datasets by solar day
[2024-04-08 13:25:01,817] {hopper.py:184} INFO - Total number of tasks: 1
CPU times: user 481 ms, sys: 9.75 ms, total: 490 ms
Wall time: 487 ms


In [13]:
tasks = [format_task(task) for task in tasks]

In [14]:
json.dump(tasks, sys.stdout)

[{"solar_day": "2016-04-05", "tile_id_x": 199, "tile_id_y": 35, "task_datasets_ids": ["9b916e21-2229-5121-8333-0a8b3736d440", "180340f9-b365-506b-b561-153af5b1490d"]}]